# Пример использования LLM Saiga/Mistral 7B, Russian Mistral-based chatbot
Автор модели [IlyaGusev](https://huggingface.co/IlyaGusev)

На базе [Mistral OpenOrca](https://huggingface.co/Open-Orca/Mistral-7B-OpenOrca).


### Требования

1. На проектном диске необходимо 15Gb свободного места
2. Минимальная конфигурация вычислительных ресурсов g1.1 (1 карта GPU Vidia V100) 

### Установим библиотеки

После выполнения ячейки перезагрузите ядро (Kernel -> Reset Kernel)

In [ ]:
%pip install accelerate==0.21.0  bitsandbytes==0.40.2   peft==0.5.0   transformers==4.34.0   sentencepiece

### Задаем путь для сохранения кешированных данных модели

In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/home/jupyter/datasphere/project/cache'

### Определяем и скачиваем модель

In [2]:
import torch
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.<s>system\n"


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]) :]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

2024-02-07 15:30:31.159126: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, torch_dtype=torch.float32, device_map="auto")
model = PeftModel.from_pretrained(model, MODEL_NAME, torch_dtype=torch.float32)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}



In [4]:
from transformers import pipeline, Conversation

In [5]:
class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE,
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{"role": "system", "content": system_prompt}]

    def add_user_message(self, message):
        self.messages.append({"role": "user", "content": message})

    def add_bot_message(self, message):
        self.messages.append({"role": "bot", "content": message})

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()

### Делаем запрос к модели

In [6]:
inputs = ["Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Баня, водка, гармонь и лосось", "Почему трава зеленая?"]
for inp in inputs:
    conversation = Conversation()
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt(tokenizer)

    output = generate(model, tokenizer, prompt, generation_config)
    print(prompt)
    print(output)
    print()
    print("==============================")
    print()

<s>system
Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.<s>system
</s><s>user
Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Баня, водка, гармонь и лосось</s><s>bot
Однажды в небольшом городке на берегу реки произошло что-то очень важное для жителей этого места. В этот день в городе открылась новая баня, которая стала самым главным событием для всех местных жителей.

Баня была построена на берегу реки, и ее стены были выполнены из камней, которые были добыты прямо на этом месте. Огромные окна давали возможность наслаждаться красивыми видами на реку и лес, окружающий город. Внутри бани было устроено несколько комнат, где можно было отдохнуть после горячего ванны или уединиться в специальной комнате для массажа.

Вот как это случилось. В один из холодных зимних дней группа друзей решила попробовать новую баню. Они пришли туда ранним утром, когда в бане еще не было никого. Вход в баню был закрыт деревянными двери, кото